# Demonstration of CnDSerializer Integration with Python Objects
This notebook demonstrates how to use the `CnDSerializer` class to serialize Python objects and visualize the resulting structure.

In [ ]:

import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

# Test the improved provider system with various Python objects
import dataclasses
from collections import namedtuple
from spytial.provider_system import CnDDataInstanceBuilder
import json

# Test with dataclass
@dataclasses.dataclass
class Employee:
    name: str
    age: int
    department: str

# Test with namedtuple
Point = namedtuple('Point', ['x', 'y'])

# Test data
test_objects = {
    "dict_example": {
        "name": "Alice",
        "age": 30,
        "hobbies": ["reading", "cycling"]
    },
    "dataclass_example": Employee("Bob", 25, "Engineering"),
    "namedtuple_example": Point(10, 20),
    "mixed_list": [1, "hello", {"nested": True}]
}

# Test each object with a fresh builder
for name, obj in test_objects.items():
    print(f"\n=== {name.upper()} ===")
    # Create a fresh builder for each object to avoid state contamination
    fresh_builder = CnDDataInstanceBuilder()
    serialized = fresh_builder.build_instance(obj)
    print(json.dumps(serialized, indent=2))
    print("-" * 50)


=== DICT_EXAMPLE ===
{
  "atoms": [
    {
      "id": "n1",
      "type": "str",
      "label": "Alice"
    },
    {
      "id": "n2",
      "type": "int",
      "label": "30"
    },
    {
      "id": "n4",
      "type": "str",
      "label": "reading"
    },
    {
      "id": "n5",
      "type": "str",
      "label": "cycling"
    },
    {
      "id": "n3",
      "type": "list",
      "label": "list[2]"
    },
    {
      "id": "n0",
      "type": "dict",
      "label": "dict{3}"
    }
  ],
  "relations": [
    {
      "id": "0",
      "name": "0",
      "types": [
        "object",
        "object"
      ],
      "tuples": [
        {
          "atoms": [
            "n3",
            "n4"
          ],
          "types": [
            "list",
            "str"
          ]
        }
      ]
    },
    {
      "id": "1",
      "name": "1",
      "types": [
        "object",
        "object"
      ],
      "tuples": [
        {
          "atoms": [
            "n3",
            "n5"
  

In [2]:
# Demonstration of the new Provider System
from spytial.provider_system import (
    CnDDataInstanceBuilder, 
    DataInstanceProvider, 
    data_provider,
    DataInstanceRegistry
)
import json

# Example: Custom provider for a specific domain object
class BankAccount:
    def __init__(self, account_number, balance, owner):
        self.account_number = account_number
        self.balance = balance
        self.owner = owner
        self._internal_id = f"acc_{account_number}"

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

# Custom provider for BankAccount objects
@data_provider(priority=15)
class BankAccountProvider(DataInstanceProvider):
    """Custom provider for BankAccount objects with special handling."""
    
    def can_handle(self, obj):
        return isinstance(obj, BankAccount)
    
    def provide_atoms_and_relations(self, obj, walker_func):
        obj_id = walker_func._get_id(obj)
        atom = {
            "id": obj_id,
            "type": "BankAccount",
            "label": f"Account #{obj.account_number}"
        }
        
        relations = []
        # Special handling: don't expose internal_id
        relations.append(("account_number", obj_id, walker_func(obj.account_number)))
        relations.append(("balance", obj_id, walker_func(obj.balance)))
        relations.append(("owner", obj_id, walker_func(obj.owner)))
        
        return atom, relations

# Test the system
test_data = {
    "accounts": [
        BankAccount("12345", 1000.0, Person("Alice", 30)),
        BankAccount("67890", 2500.0, Person("Bob", 25))
    ],
    "total_accounts": 2
}

builder = CnDDataInstanceBuilder()
result = builder.build_instance(test_data)

print("=== CUSTOM PROVIDER SYSTEM DEMO ===")
print(json.dumps(result, indent=2))

=== CUSTOM PROVIDER SYSTEM DEMO ===
{
  "atoms": [
    {
      "id": "n3",
      "type": "str",
      "label": "12345"
    },
    {
      "id": "n4",
      "type": "float",
      "label": "1000.0"
    },
    {
      "id": "n6",
      "type": "str",
      "label": "Alice"
    },
    {
      "id": "n7",
      "type": "int",
      "label": "30"
    },
    {
      "id": "n5",
      "type": "Person",
      "label": "Person"
    },
    {
      "id": "n2",
      "type": "BankAccount",
      "label": "Account #12345"
    },
    {
      "id": "n9",
      "type": "str",
      "label": "67890"
    },
    {
      "id": "n10",
      "type": "float",
      "label": "2500.0"
    },
    {
      "id": "n12",
      "type": "str",
      "label": "Bob"
    },
    {
      "id": "n13",
      "type": "int",
      "label": "25"
    },
    {
      "id": "n11",
      "type": "Person",
      "label": "Person"
    },
    {
      "id": "n8",
      "type": "BankAccount",
      "label": "Account #67890"
    },
    {

In [3]:
# Example: Provider that handles numpy arrays (if available)
try:
    import numpy as np
    
    @data_provider(priority=12)
    class NumpyArrayProvider(DataInstanceProvider):
        def can_handle(self, obj):
            return isinstance(obj, np.ndarray)
        
        def provide_atoms_and_relations(self, obj, walker_func):
            obj_id = walker_func._get_id(obj)
            atom = {
                "id": obj_id,
                "type": "ndarray",
                "label": f"array{obj.shape}"
            }
            
            relations = []
            # For small arrays, expose individual elements
            if obj.size <= 10:
                for i, val in enumerate(obj.flat):
                    vid = walker_func(val.item())  # Convert to Python type
                    relations.append((f"element_{i}", obj_id, vid))
            else:
                # For large arrays, just show metadata
                relations.append(("shape", obj_id, walker_func(obj.shape)))
                relations.append(("dtype", obj_id, walker_func(str(obj.dtype))))
            
            return atom, relations
    
    # Test with numpy array
    arr = np.array([1, 2, 3, 4, 5])
    numpy_result = builder.build_instance({"data": arr, "description": "sample data"})
    print("\n=== NUMPY ARRAY EXAMPLE ===")
    print(json.dumps(numpy_result, indent=2))
    
except ImportError:
    print("\n=== NUMPY NOT AVAILABLE ===")
    print("Install numpy to see array provider example")

# Example: Overriding default behavior for specific types
@data_provider(priority=20)  # Higher priority than default dict provider
class SpecialDictProvider(DataInstanceProvider):
    """Handle dictionaries with 'special_' prefix differently."""
    
    def can_handle(self, obj):
        return (isinstance(obj, dict) and 
                any(key.startswith('special_') for key in obj.keys()))
    
    def provide_atoms_and_relations(self, obj, walker_func):
        obj_id = walker_func._get_id(obj)
        atom = {
            "id": obj_id,
            "type": "SpecialDict",
            "label": f"Special({len(obj)})"
        }
        
        relations = []
        for k, v in obj.items():
            vid = walker_func(v)
            # Remove 'special_' prefix from relation names
            rel_name = k.replace('special_', '') if k.startswith('special_') else k
            relations.append((rel_name, obj_id, vid))
        
        return atom, relations

# Test special dict
special_data = {
    "normal_key": "normal_value",
    "special_config": {"setting1": True, "setting2": 42},
    "special_metadata": {"version": "1.0", "author": "system"}
}

special_result = builder.build_instance(special_data)
print("\n=== SPECIAL DICT PROVIDER EXAMPLE ===")
print(json.dumps(special_result, indent=2))


=== NUMPY ARRAY EXAMPLE ===
{
  "atoms": [
    {
      "id": "n2",
      "type": "int",
      "label": "1"
    },
    {
      "id": "n3",
      "type": "int",
      "label": "2"
    },
    {
      "id": "n4",
      "type": "int",
      "label": "3"
    },
    {
      "id": "n5",
      "type": "int",
      "label": "4"
    },
    {
      "id": "n6",
      "type": "int",
      "label": "5"
    },
    {
      "id": "n1",
      "type": "ndarray",
      "label": "array(5,)"
    },
    {
      "id": "n7",
      "type": "str",
      "label": "sample data"
    },
    {
      "id": "n0",
      "type": "dict",
      "label": "dict{2}"
    }
  ],
  "relations": [
    {
      "id": "element_0",
      "name": "element_0",
      "types": [
        "object",
        "object"
      ],
      "tuples": [
        {
          "atoms": [
            "n1",
            "n2"
          ],
          "types": [
            "ndarray",
            "int"
          ]
        }
      ]
    },
    {
      "id": "elem

In [4]:
# Define a custom class
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

# Create an instance of the custom class
person = Person("Bob", 25)

# Use the provider system to serialize the custom class instance
builder = CnDDataInstanceBuilder()
custom_serialized_output = builder.build_instance(person)

print("=== CUSTOM CLASS SERIALIZATION ===")
print(json.dumps(custom_serialized_output, indent=2))


=== CUSTOM CLASS SERIALIZATION ===
{
  "atoms": [
    {
      "id": "n1",
      "type": "str",
      "label": "Bob"
    },
    {
      "id": "n2",
      "type": "int",
      "label": "25"
    },
    {
      "id": "n0",
      "type": "Person",
      "label": "Person"
    }
  ],
  "relations": [
    {
      "id": "name",
      "name": "name",
      "types": [
        "object",
        "object"
      ],
      "tuples": [
        {
          "atoms": [
            "n0",
            "n1"
          ],
          "types": [
            "Person",
            "str"
          ]
        }
      ]
    },
    {
      "id": "age",
      "name": "age",
      "types": [
        "object",
        "object"
      ],
      "tuples": [
        {
          "atoms": [
            "n0",
            "n2"
          ],
          "types": [
            "Person",
            "int"
          ]
        }
      ]
    }
  ]
}


In [5]:
# Demonstration of the new diagram() function
from spytial import diagram

# Example data with nested structure
demo_data = {
    "users": [
        {"name": "Alice", "age": 30, "skills": ["Python", "JavaScript"]},
        {"name": "Bob", "age": 25, "skills": ["Java", "C++"]}
    ],
    "projects": {
        "web_app": {"lead": "Alice", "status": "active"},
        "mobile_app": {"lead": "Bob", "status": "planning"}
    },
    "stats": {"total_users": 2, "active_projects": 1}
}

# Method 1: Build instance and show
builder = CnDDataInstanceBuilder()
data_instance = builder.build_instance(demo_data)
print("=== INTERACTIVE VISUALIZATION ===")
print("Displaying interactive graph below...")

# Show the visualization inline (in Jupyter) 
diagram(data_instance, method="inline")



=== INTERACTIVE VISUALIZATION ===
Displaying interactive graph below...


In [6]:
# Different ways to display visualizations

# Create some test data
simple_example = {
    "name": "Example",
    "items": [1, 2, 3],
    "metadata": {"version": "1.0", "active": True}
}

print("=== DISPLAY METHODS ===")
print("1. Inline (Jupyter notebook):")
print("   diagram(data_instance, method='inline')")
print()
print("2. Browser (opens new tab):")
print("   diagram(data_instance, method='browser')")
print()
print("3. Save to file:")
print("   diagram(data_instance, method='file')")
print()
print("4. Quick one-liner:")
print("   quick_diagram(my_object)")

# Example: Save to file (commented out to avoid creating files during demo)
# file_path = diagram(builder.build_instance(simple_example), method="file")
# print(f"Saved visualization to: {file_path}")

# Show the structure as JSON for reference
print("\n=== DATA STRUCTURE ===")
instance = builder.build_instance(simple_example)
print(json.dumps(instance, indent=2))

=== DISPLAY METHODS ===
1. Inline (Jupyter notebook):
   diagram(data_instance, method='inline')

2. Browser (opens new tab):
   diagram(data_instance, method='browser')

3. Save to file:
   diagram(data_instance, method='file')

4. Quick one-liner:
   quick_diagram(my_object)

=== DATA STRUCTURE ===
{
  "atoms": [
    {
      "id": "n1",
      "type": "str",
      "label": "Example"
    },
    {
      "id": "n3",
      "type": "int",
      "label": "1"
    },
    {
      "id": "n4",
      "type": "int",
      "label": "2"
    },
    {
      "id": "n5",
      "type": "int",
      "label": "3"
    },
    {
      "id": "n2",
      "type": "list",
      "label": "list[3]"
    },
    {
      "id": "n7",
      "type": "str",
      "label": "1.0"
    },
    {
      "id": "n8",
      "type": "bool",
      "label": "True"
    },
    {
      "id": "n6",
      "type": "dict",
      "label": "dict{2}"
    },
    {
      "id": "n0",
      "type": "dict",
      "label": "dict{3}"
    }
  ],
  "rela

In [7]:
# Test the new show function that takes raw Python objects
from spytial.visualizer import diagram

print("Testing the new diagram() function with test_data:")
print("This will serialize the object using the provider system and display it inline.")

# Test with our complex test data
diagram(test_data, method="inline")

Testing the new diagram() function with test_data:
This will serialize the object using the provider system and display it inline.


In [8]:
# Test with a simpler object
from spytial.visualizer import diagram

simple_obj = {
    "name": "Alice",
    "age": 30,
    "hobbies": ["reading", "cycling"],
    "address": {
        "street": "123 Main St",
        "city": "Anytown"
    }
}

print("Testing with a simple object:")
diagram(simple_obj, method="inline")

Testing with a simple object:


In [9]:
# Demonstrate different display methods
print("=== Different Display Methods ===")

# 1. Save to file
file_path = diagram(simple_obj, method="file")
print(f"✅ Saved visualization to: {file_path}")

# 2. Browser method (creates temp file, doesn't auto-open)
browser_path = diagram(simple_obj, method="browser", auto_open=False)
print(f"✅ Created browser file at: {browser_path}")

print("\n💡 The diagram() function provides three methods:")
print("   - 'inline': Display in Jupyter notebook (default)")
print("   - 'browser': Open in web browser")  
print("   - 'file': Save to cnd_visualization.html file")

=== Different Display Methods ===
Visualization saved to: /Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/cnd_visualization.html
✅ Saved visualization to: /Users/siddharthaprasad/Desktop/SpatialRefinement/cnd-py/cnd_visualization.html
✅ Created browser file at: /var/folders/80/rtptthbx3zq0tb06wwzmck_40000gq/T/tmpy9mk9dpd.html

💡 The diagram() function provides three methods:
   - 'inline': Display in Jupyter notebook (default)
   - 'browser': Open in web browser
   - 'file': Save to cnd_visualization.html file
